# European Union Energy Map

Dashboard with interactive map visualizing Eurostat data regarding renewable energy developments in the European Union.

Data is pulled from [Eurostat](https://ec.europa.eu/eurostat/databrowser/explore/all/envir?lang=en&subtheme=nrg&display=list&sort=category).

**GitHub**: [https://github.com/kuranez/EU-Energy-Map](https://github.com/kuranez/EU-Energy-Map)

# Data Info

**Renewable Energy Dataset**

---

This dataset provides information on the **share of energy from renewable sources** across various European countries. It originates from **Eurostat**, the statistical office of the European Union, and includes annual data on renewable energy consumption and production.

- **Filename:** **`nrg_ind_ren_linear.csv`**
- **Title:** **Share of energy from renewable sources**
- **Source:** [Eurostat - Share of Renewable Energy](https://ec.europa.eu/eurostat/databrowser/view/nrg_ind_ren/default/table?lang=en)
- **Time Range:** 2004–2022
- **Geographical Coverage:** European countries, EU aggregates, and select neighboring states
- **Key Indicators:** Energy type, renewable percentage, country codes and years

**Geoprahical data**

---

This dataset contains the **administrative boundaries of European countries** in GeoJSON format, suitable for web-based mapping and spatial analysis. It is sourced from **GISCO - Eurostat**, ensuring accuracy and alignment with official European statistics.

- **Filename:** **`europe.geojson`**
- **Source:** [Countries - GISCO - Eurostat](https://ec.europa.eu/eurostat/web/gisco/geodata/administrative-units/countries)
- **Year:** 2024
- **File format:** GeoJSON
	A lightweight and widely supported format for web mapping applications.
- **Geometry Type:** Polygons (RG)
	Provides full administrative boundaries as polygons, making it ideal for accurately displaying country borders in mapping applications.
- **Scale:** 20M
    A 1:20,000,000 scale offers a balanced level of detail while keeping file size manageable. This is well-suited for web applications where performance is a priority.
- **Coordinate Reference System:** EPSG: 4326
	Uses the WGS 84 latitude/longitude system, the standard for global web mapping. This CRS ensures compatibility with most web mapping libraries, including Leaflet, Mapbox, and OpenLayers, making it the best choice for interactive maps.

# Notebook Structure

```plaintext
EU Energy Map v.1.0 : Workflow Diagram
├── 1. Import
│   ├── a. Standard Library Modules
│   ├── b. Dashboard and WebApp - Panel
│   ├── c. Data Analysis - Pandas and Geopandas
│   ├── d. Data Visualization - Plotly Ecosystem
│
├── 2. Settings
│   ├── a. Set Panel extension
│   ├── b. Set Plotly renderer
│   ├── c. Set MapBox Access Token
│
├── 3. Methods
│   ├── a. Country Flags
│   ├── b. Load Data
│   ├── c. Filter Data
│   ├── d. Data Calculations
│   ├── e. Create Map
│   ├── f. Create Charts
│   ├── g. Create Widgets
│
├── 4. Execute
│   ├── a. Widgets
│   ├── b. Sidebar
│   ├── c. Main Panel Components
│       ├── DataTable
│       ├── Map
│       ├── Bar Charts
│
├── 5. Dashboard Creation
│   ├── a. Create Components
│   ├── b. Define Update Functions
│   ├── c. Bind Widgets to Update Functions
│
├── 6. Dashboard Layout
│   ├── a. Create Panes
│   ├── b. Create Tabs
│   ├── c. Create Layout
│
└── 7. Serve Dashboard
```

# I. Import

- **a.** **Standard Library Modules** (Built-in Python modules)
- **b.** **Interactive Dashboard** and **Web App Development**
- **c.** **Data Manipulation** and **Geospatial Analysis**
- **d.** **Data Visualization** (Plotly Ecosystem)

##### [Show/Hide Code]

In [67]:
# a. Standard Library Modules (Built-in Python modules)
import os
import json

# b. Interactive Dashboard and Web App Development
import panel as pn

# c. Data Manipulation and Geospatial Analysis 
import pandas as pd
import geopandas as gpd

# d. Data Visualization (Plotly Ecosystem)
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as pc
from plotly.colors import sample_colorscale

# II. Settings

- **a.** Set **Panel** extension
- **b.** Set **Plotly** renderer
- **c.** Set **MapBox**-Access-Token

##### [Show/Hide Code]

In [2]:
# a. Set up Panel extension
pn.extension('tabulator', 'plotly', design='material', sizing_mode='stretch_width')

# b. Set Plotly renderer to JupyterLab
# pio.renderers.default = "jupyterlab"

# c. Set MapBox-Access-Token
MAPBOX_TOKEN = 'pk.eyJ1Ijoia3VyYW5leiIsImEiOiJjbTJmMjI0d2kwNDVxMnFzYXNldnc1N2VsIn0.t11TYpF2QBdid-hQfW8mig'

# III. Methods

### a. Country Flags : `iso2_to_flag()` & `add_country_flags()`

In [3]:
@pn.cache
def iso2_to_flag(iso2_code):
    """
    Function to convert ISO2 country code to flag emoji
    """
    return chr(0x1F1E6 + ord(iso2_code[0]) - ord('A')) + chr(0x1F1E6 + ord(iso2_code[1]) - ord('A'))


In [4]:
@pn.cache
def add_country_flags(data):
    """
    Add country flags to the dataset using country codes.
    """
    data['Flag'] = data['Code'].apply(iso2_to_flag)
    return data

### b. Sort Columns : `sort_columns()`

##### **NEW** Helper Function **!**

In [5]:
@pn.cache
def sort_columns(data, column_order):
    """
    Sort the DataFrame columns in the specified order.
    """
    return data[column_order]

### c. Load Data : `load_data()`

##### **NEW** Combined Loading Method **!**

In [6]:
def load_data():
    """Load energy data and European geojson data."""
    data = pd.read_csv('./data/nrg_ind_ren_linear.csv')
    europe_gdf = gpd.read_file("./geo/europe.geojson")

    merged_data = europe_gdf.merge(data, left_on='CNTR_ID', right_on='geo')

    merged_data.rename(columns={
        'nrg_bal': 'Energy Type', 'TIME_PERIOD': 'Year',
        'OBS_VALUE': 'Renewable Percentage', 'geo': 'Code',
        'NAME_ENGL': 'Country'
    }, inplace=True)

    energy_type_map = {
        'REN': 'Renewable Energy Total',
        'REN_ELC': 'Renewable Electricity',
        'REN_HEAT_CL': 'Renewable Heating and Cooling',
        'REN_TRA': 'Renewable Energy in Transport'
    }
    merged_data['Energy Type'] = merged_data['Energy Type'].replace(energy_type_map)

    merged_data.drop(columns=['LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
    merged_data[['Year', 'Renewable Percentage']] = merged_data[['Year', 'Renewable Percentage']].apply(pd.to_numeric)
    merged_data['Renewable Percentage'] = merged_data['Renewable Percentage'].round(1)
    
    merged_data['Flag'] = merged_data['Code'].apply(iso2_to_flag)
    merged_data = add_country_flags(merged_data)

    final_columns = ['Code', 'Flag', 'Country', 'Energy Type', 'Renewable Percentage', 'Year', 'geometry']
    final_data = sort_columns(merged_data, final_columns)
    
    return final_data

### Test

In [7]:
# load_data()

### Substeps (for Debugging)

##### [Show/Hide Code: 1] - **Load**

In [8]:
# # Load data
# data = pd.read_csv('./data/nrg_ind_ren_linear.csv')
# europe_gdf = gpd.read_file("./geo/europe.geojson")

# # Display data
# data.head()
# europe_gdf.head()

##### [Show/Hide Code: 2] - **Merge**

In [9]:
# # Merge data
# merged_data = europe_gdf.merge(data, left_on='CNTR_ID', right_on='geo')

# # Display merged data
# merged_data.head()

##### [Show/Hide Code: 3.a] - **Rename Columns**

In [10]:
# merged_data.rename(columns={
#     'nrg_bal': 'Energy Type', 
#     'TIME_PERIOD': 'Year',
#     'OBS_VALUE': 'Renewable Percentage', 
#     'geo': 'Code',
#     'NAME_ENGL': 'Country'
# }, inplace=True)

##### [Show/Hide Code: 3.b] - **Rename Entries**

In [11]:
# energy_type_map = {
#     'REN': 'Renewable Energy Total',
#     'REN_ELC': 'Renewable Electricity',
#     'REN_HEAT_CL': 'Renewable Heating and Cooling',
#     'REN_TRA': 'Renewable Energy in Transport'
# }

In [12]:
# merged_data['Energy Type'] = merged_data['Energy Type'].replace(energy_type_map)

##### [Show/Hide Code: 4] - **Drop Columns**

In [13]:
 # merged_data.drop(columns=['LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'], inplace=True)

##### [Show/Hide Code: 5] - **Number Operations**

In [14]:
# merged_data[['Year', 'Renewable Percentage']] = merged_data[['Year', 'Renewable Percentage']].apply(pd.to_numeric)
# merged_data['Renewable Percentage'] = merged_data['Renewable Percentage'].round(1)

##### [Show/Hide Code: 6] - **Country Flags**

In [15]:
# merged_data['Flag'] = merged_data['Code'].apply(iso2_to_flag)
# merged_data = add_country_flags(merged_data)

##### [Show/Hide Code: 7] - **Order Columns**

In [16]:
# final_columns = [
#     'Code', 'Flag', 'Country', 'Energy Type', 'Renewable Percentage', 'Year', 'geometry'
# ]

### d. Normalize Color Scale : `get_plotly_color()` 

In [17]:
def get_plotly_color(value, vmin, vmax):
    """ Normalize a value to the Viridis colormap range. """
    normalized_value = (value - vmin) / (vmax - vmin)  # Scale between 0 and 1
    return pc.sample_colorscale("viridis", normalized_value)[0]  # Get color

## IV. Data Filters

In [18]:
# eu_countries = {"AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "EL", "HU", "IE", "IT", "LV", "LT",
#                 "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"}

In [19]:
# @pn.cache
# def filter_by_country_codes(dataset, country_codes):
#     """
#     Filters the dataset based on a list of country codes.
#     """
#     return dataset[dataset['Country Code'].isin(country_codes)]

In [20]:
# Filter by Year & Country : for testing/initial display
selected_year = 2022 # 
selected_country = "Germany"

In [21]:
def filter_data(data):
    """Filter and structure dataset for visualization."""
    eu_countries = {"AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT",
                    "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"}
    df_renewable = data[(data['Energy Type'] == 'Renewable Energy Total') & data['Code'].isin(eu_countries)]
    df_eu_total = df_renewable.groupby('Year', as_index=False)['Renewable Percentage'].mean().round(1)
    df_year = df_renewable[df_renewable['Year'] == selected_year]
    df_country = df_renewable[df_renewable['Country'] == selected_country]
    return df_renewable, df_eu_total, df_year, df_country

### Test

In [22]:
# Return Data
# data = load_data()

In [23]:
# filtered_data = filter_data(data)

In [24]:
# data.info()

In [25]:
# df_renewable

In [26]:
# df_eu_total

In [27]:
# df_year

In [28]:
# df_country

### Substeps (for Debugging)

##### **Step 1:** Filter by EU countries [Show/Hide Code]

In [29]:
# # Step 1: Filter by EU countries
# eu_countries = [
#     "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", 
#     "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"
# ]

# eu_data = merged_data[data['Code'].isin(eu_countries)]

##### **Step 2:** Filter by Energy Type [Show/Hide Code]

In [30]:
# # Step 2: Filter by Energy Type 
# df_renewable = data[
#                     (data['Energy Type'] == 'Renewable Energy Total') &
#                     (data['Code'].isin(eu_countries))]


##### **Step 3:** **EU Total** Average Renewable Energy Percentage[Show/Hide Code]

In [31]:
# # Step 3: EU Total Average Renewable Energy Percentage
# # - Group by year and calculate the average renewable energy percentage for the EU total
# df_eu_total = df_renewable.groupby('Year', as_index=False)['Renewable Percentage'].mean()

##### **Step 4:** Filter by Year & Country[Show/Hide Code]

In [32]:
# # Step 4: Filter by Year & Country
# df_year = df_renewable[df_renewable['Year'] == selected_year]
# df_country = df_renewable[df_renewable['Country'] == selected_country]

##### **Test** [Show/Hide Code]

In [33]:
# eu_data.head()

# IV. Main : Load and preprocess data

In [34]:
data = load_data()
df_renewable, df_eu_total, df_year, df_country = filter_data(data)
selected_year, selected_country = 2022, "Germany"
df_year = df_renewable[df_renewable['Year'] == selected_year]
df_country = df_renewable[df_renewable['Country'] == selected_country]

# Get min and max energy values for consistent scaling
vmin = df_renewable["Renewable Percentage"].min()
vmax = df_renewable["Renewable Percentage"].max()

# Apply color normalization
# df_renewable["color"] = df_renewable["Renewable Percentage"].apply(lambda x: get_scaled_color(x, vmin, vmax))
# df_renewable.loc[:, "color"] = df_renewable["Renewable Percentage"].apply(lambda x: get_plotly_color(x, vmin, vmax))
df_renewable = df_renewable.copy()  # Creates a new independent DataFrame
df_renewable["color"] = df_renewable["Renewable Percentage"].apply(lambda x: get_plotly_color(x, vmin, vmax))

# V. Dashboard

### a. Input Widgets : `year_slider`& `country_selection`

In [35]:
# # Dynamically determine year range
# start_year = df_renewable['Year'].min()
# end_year = df_renewable['Year'].max()

In [36]:
# # Map countries to flags
# country_flag_map = dict(zip(df_renewable['Country'], df_renewable['Flag']))

# # Create country selection widget with country names only
# unique_countries = sorted(country_flag_map.keys())

In [37]:
# Widgets
year_slider = pn.widgets.IntSlider(name='Year', start=2004, end=2022, step=1, value=selected_year)
country_selection = pn.widgets.Select(name='Country', options=df_renewable['Country'].unique().tolist(), value=selected_country)

### e. Create Map : `create_chloropleth_map()` 

In [38]:
@pn.cache
def create_choropleth_map(df_year):
    """Generate a Choropleth map of renewable energy percentages."""

    # # Map EU total average to a color in the Viridis scale
    # eu_total_avg = df_year['Renewable Percentage'].mean()
    # scaled_color = get_scaled_color(eu_total_avg)
    
    fig = go.Figure(go.Choroplethmapbox(
        geojson=json.load(open('./geo/europe.geojson')),
        locations=df_year['Code'],
        z=df_year['Renewable Percentage'],
        colorscale="Viridis",
        marker_opacity=0.8, marker_line_width=0.5,
        featureidkey="properties.CNTR_ID"
    ))
    fig.update_layout(mapbox_accesstoken=MAPBOX_TOKEN, mapbox_style="carto-positron", mapbox_zoom=3,
                      mapbox_center={"lat": 54, "lon": 15}, margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig

In [39]:
# Dynamic data update
@pn.depends(year_slider.param.value)
def update_map(year):
    filtered_df = df_renewable[df_renewable['Year'] == year]
    fig = create_choropleth_map()
    fig.data[0].update(
        locations=filtered_df['Country Code'],
        z=filtered_df['Renewable Percentage'],
        hovertext=filtered_df['Renewable Percentage']
    )
    return pn.pane.Plotly(fig)

##### [Show/Hide Code] - **Map**

In [40]:
# @pn.cache
# # Create Choropleth map function
# def create_choropleth_map(df_year):
#     fig_map = go.Figure(go.Choroplethmapbox(
#         geojson=europe_map,
#         locations=df_year['Code'],
#         z=df_year['Renewable Percentage'],
#         colorscale="Viridis",
#         zmin=0, zmax=100,
#         marker_opacity=0.8, marker_line_width=0.5,
#         featureidkey="properties.CNTR_ID",
#         hoverinfo='location+z', hovertext=df_year['Renewable Percentage']
#     ))
    
#     fig_map.update_layout(
#         mapbox_accesstoken=mapbox_token,
#         mapbox_style="carto-positron",
#         mapbox_zoom=3,
#         mapbox_center={"lat": 54, "lon": 15},
#         title="Renewable Energy as Percentage by Country in 2022",
#         margin={"r": 0, "t": 0, "l": 0, "b": 0},
#         geo=dict(scope='europe', showlakes=False)
#     )
#     return fig_map

## e. Create Charts : `create_bar_chart year()` & `create_bar_chart_country()` 

##### [Show/Hide Code] - **Year Chart**

In [72]:
import plotly.express as px
from plotly.colors import sample_colorscale

def get_scaled_color(value, vmin=0, vmax=100):
    """Get a color from the Viridis color scale based on the scaled value."""
    norm = (value - vmin) / (vmax - vmin)
    color = px.colors.sample_colorscale("Viridis", [norm])[0]
    return color

# Create bar chart by year for all countries
@pn.cache
@pn.depends(year_slider.param.value)
def create_bar_chart_year(df_year, year):
    # Sort dataframe by renewable percentage in ascending order
    df_year = df_year.sort_values(by='Renewable Percentage')

    # # Calculate EU total average renewable percentage for 2022
    eu_total_avg = df_year['Renewable Percentage'].mean()
    
    # # # Map EU total average to a color in the Viridis scale
    # scaled_color = get_scaled_color(eu_total_avg)
    color_scale = px.colors.sequential.Viridis
    normalized_avg = eu_total_avg / 100  # Normalize average to a 0-1 range
    scaled_color = sample_colorscale(color_scale, normalized_avg)[0]
    
    # Create bar trace (main layer)
    bar_trace = go.Bar(
        x=df_year['Country'],
        y=df_year['Renewable Percentage'],
        marker=dict(
            color=df_year['Renewable Percentage'],
            coloraxis='coloraxis',  # Link to coloraxis
        ),
        name="",  # Fix to hide trace info
        hovertemplate="Country: %{customdata[0]}"
                      "<b>%{customdata[1]}</b><br>"
                      "Renewable Percentage: <b>%{y:.1f}%</b>",
        customdata=df_year[['Flag', 'Country']].values,  # Attach custom data
        showlegend=False,
    )
    
    # Add scatter trace for EU Total Average (border layer)
    scatter_trace_border = go.Scatter(
        x=df_year['Country'],  # Use all countries to make the line span across
        y=[eu_total_avg] * len(df_year),  # Duplicate average value for each country
        mode="lines",
        line=dict(dash="solid", color="rgba(255, 255, 255, 0.7)", width=4),
        hoverinfo="skip",  # Suppress hover for border layer
        showlegend=False,
    )
    
    # Add scatter trace for EU Total Average (main layer)
    scatter_trace = go.Scatter(
        x=df_year['Country'],  # Use all countries to make the line span across
        y=[eu_total_avg] * len(df_year),  # Duplicate average value for each country
        mode="lines",
        line=dict(dash="solid", color=scaled_color, width=2),
        hovertemplate="🇪🇺 EU Total Average:<b> %{y:.1f}%</b>",
        name="",  # Fix to suppress showing trace info
        showlegend=False,
    )

    # Create figure
    fig = go.Figure(data=[bar_trace, scatter_trace_border, scatter_trace])

    # Update layout
    fig.update_layout(
        title=f"Renewable Energy Percentage by Country in {year}",
        xaxis=dict(title=None),
        yaxis=dict(title="Renewable Energy (%)"),
        coloraxis=dict(  # Define coloraxis for color bar
            colorscale='Viridis',
            cmin=0,
            cmax=100,
            colorbar=dict(
                orientation="v",
                title=None,
                tickvals=[0, 20, 40, 60, 80, 100],
                ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
            ),
        ),
        margin={"t": 50, "b": 50, "l": 50, "r": 50},
        height=450,
    )

    return fig

In [73]:
# create_bar_chart_year(df_year, selected_year)

##### [Show/Hide Code] - **Country Chart**

In [74]:
import plotly.express as px
# Create Country Chart
@pn.cache
@pn.depends(country_selection.param.value)
def create_bar_chart_country(df_eu_total, df_country, country):

    def get_scaled_color(value, vmin=0, vmax=100):
        """Scale the color based on the renewable percentage."""
        normalized_value = (value - vmin) / (vmax - vmin)
        color_scale = px.colors.sequential.Viridis
        color = px.colors.sample_colorscale(color_scale, [normalized_value])[0]
        return color
    
    # Calculate the average renewable energy percentage for all years (used for annotation)
    eu_total_avg = df_eu_total['Renewable Percentage'].mean()
    
    # # Map EU total average to a color in the Viridis scale
    scaled_color = get_scaled_color(eu_total_avg)
    color_scale = px.colors.sequential.Viridis
    normalized_avg = eu_total_avg / 100  # Normalize average to a 0-1 range
    scaled_color = px.colors.sample_colorscale(color_scale, normalized_avg)[0]

    # Create the figure with Plotly
    fig = go.Figure()

    # Add the line trace for EU total renewable energy percentage over the years
    fig.add_trace(go.Scatter(
        x=df_eu_total['Year'], 
        y=df_eu_total['Renewable Percentage'], 
        mode='lines+markers',
        line=dict(color=scaled_color, width=3),
        marker=dict(
            size=6,
            color=df_eu_total['Renewable Percentage'],  # Map the renewable percentage to color
            colorscale='Viridis',
            coloraxis="coloraxis",
        ),
        hovertemplate="🇪🇺 %{x}: <b>%{y:.1f}%</b>",
        name="EU Total Average",
    ))

    # Add the bar trace for the selected country's renewable energy percentage
    fig.add_trace(go.Bar(
        x=df_country['Year'], 
        y=df_country['Renewable Percentage'],
        marker=dict(
            color=df_country['Renewable Percentage'],  # Map the renewable percentage to color
            colorscale='Viridis',
            coloraxis="coloraxis",
        ),
        customdata=df_country[['Flag', 'Country']].values,  # Attach custom data
        hovertemplate="Country: %{customdata[0]} "
                      "<b>%{customdata[1]}</b><br>"
                      "Year: <b>%{x}</b><br>"
                      "Renewable Percentage: <b>%{y:.1f}%</b>",
        name="",
        showlegend=False,
    ))

    # Define the layout with legend at the bottom
    fig.update_layout(
        title=f"Renewable Energy Percentage ({country}, 2004-2022)",
        xaxis=dict(
            title="Year",
            showgrid=False,
            showline=True,
            tickmode="linear",
            tick0=2005,
            dtick=5,
            range=[2003.5, 2022.5],
        ),
        yaxis=dict(
            title="Renewable Energy (%)",
        ),
        coloraxis=dict(
            colorscale='Viridis',
            cmin=0,
            cmax=100,
            colorbar=dict(
                orientation="v",
                tickvals=[0, 20, 40, 60, 80, 100],
                ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
            ),
        ),
        height=450,
        margin={"t": 50, "b": 50, "l": 50, "r": 50},
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1-0.01,
            xanchor="left",
            x=0+0.01
        ),
    )

    return fig

In [75]:
# create_bar_chart_country(df_eu_total, df_country, selected_country)

# V. Dashboard 

##### [Show/Hide Code] - **Table**

In [76]:
# Display data in Tabulator widget
fig_table = pn.widgets.Tabulator(
    df_renewable[['Country', 'Code', 'Flag', 'Energy Type', 'Renewable Percentage', 'Year']],
    pagination="remote",          # Enables pagination
    page_size=10,                 # Sets the number of rows per page
    show_index=False              # Hides the index column
)

##### [Show/Hide Code] - **Sidebar**

In [77]:
# Define Markdown content
sidebar_content = """
#### About
Explore renewable energy developments in the European Union. Data is sourced from [Eurostat](https://ec.europa.eu/eurostat/databrowser/explore/all/envir?lang=en&subtheme=nrg&display=list&sort=category).

#### Project Page on GitHub
[https://github.com/kuranez/EU-Energy-Map](https://github.com/kuranez/EU-Energy-Map) \
"""

sidebar_pane = pn.pane.Markdown(sidebar_content)

##### [Show/Hide Code] - **Widgets**

In [78]:
@pn.depends(year_slider.param.value, watch=True)
def filter_by_year(year):
    return df_renewable[df_renewable['Year'] == year]

In [79]:
# # Callback to update filtered data
@pn.depends(country_selection.param.value, watch=True)
def filter_by_country(country):
    return df_renewable[df_renewable['Country'] == country]

# # Combine selected country's flag for display
# @pn.depends(country_selection.param.value)
# def display_country_flag(country):
#     return f"{country_flag_map[country]} {country}"

##### [Show/Hide Code] - **Component Creation**

In [80]:
# Step 1: Create Map
eu_map = create_choropleth_map(df_year)

# Step 2: Create Charts
year = selected_year
fig_by_year = create_bar_chart_year(df_year, year)

country = selected_country
fig_by_country = create_bar_chart_country(df_eu_total, df_country, country)

##### [Show/Hide Code] - **Update Selection**

In [81]:
# Map and bar chart update functions
@pn.cache
def update_map(year):
    df_selected_year = df_renewable[df_renewable['Year'] == year]
    return create_choropleth_map(df_selected_year)
# Callback to filter data by selected year

@pn.cache
def update_year_chart(year):
    df_selected_year = df_renewable[df_renewable['Year'] == year].sort_values(by='Renewable Percentage')
    return create_bar_chart_year(df_selected_year, year)

@pn.cache
def update_country_chart(country):
    df_selected_country = df_renewable[df_renewable['Country'] == country]
    return create_bar_chart_country(df_eu_total, df_selected_country, country)

def update_country_chart(selected_value):
    # Extract the country name from the selection
    # selected_country = selected_value.split(" ", 1)[1]
    
    # Filter the dataframe for the selected country
    df_selected_country = df_renewable[df_renewable['Country'] == selected_country]
    
    # Create the bar chart
    return create_bar_chart_country(df_eu_total, df_selected_country, selected_country)


##### [Show/Hide Code] - **Bind Selection**

In [82]:
# Step 1: Bind year selection
interactive_map = pn.bind(update_map, year_slider)

# Step 2: Bind year selection to chart
interactive_bar_chart_year = pn.bind(update_year_chart, year_slider)

# Step 3: Bind country selection
interactive_bar_chart_country = pn.bind(update_country_chart, country_selection.param.value)

##### [Show/Hide Code] - **Panes**

In [83]:
filter_year_pane = pn.Column(year_slider, pn.pane.Plotly(interactive_bar_chart_year,))
filter_country_pane = pn.Column(country_selection, pn.pane.Plotly(interactive_bar_chart_country))
table_pane = pn.Column(fig_table)

##### [Show/Hide Code] - **Tabs**

In [86]:
tabs = pn.Tabs(
    ('Filter by Year', filter_year_pane),
    ('Filter by Country', filter_country_pane),
    ('Explore Data', table_pane),
)

tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'5b2b001a-e5e9-4cd9-9f57-f7fdebaa4bb3': {'version…

##### [Show/Hide Code] - **Layout**

In [87]:
layout = pn.Column(
    pn.pane.Plotly(interactive_map, sizing_mode='stretch_width'),
    tabs,
    sizing_mode='stretch_width'
)

#layout

##### [Show/Hide Code] - **Serve Dashboard**

In [88]:
pn.template.FastListTemplate(
    title="EU Energy Map", sidebar=[sidebar_pane],
    logo="./img/eu-svgrepo-com-256px.svg",
    main=[layout],
).servable();